<a href="https://colab.research.google.com/github/AliTavakoli2001/Deep-Learning-Project/blob/main/Autoencoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Add the necessary Frameworks and Libraries**

In [ ]:
import tensorflow as TF
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape
from tensorflow.keras.datasets import mnist
import numpy as NP
import matplotlib.pyplot as PLT

# **Load Mnist Data**

> **Why unlabeled data?**

In an autoencoder, we only need unlabeled data because it is an unsupervised learning model. The goal of an autoencoder is to learn the underlying patterns of the data and reconstruct the input data without requiring any labels. It focuses on minimizing the reconstruction error rather than predicting labels.



In [ ]:
(x_train, _), (x_test, _) = mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
